<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv8/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov11")

In [ ]:
import albumentations as A
import cv2
import os
import numpy as np

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=40, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

def read_label(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            if line.strip():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])
    return np.array(labels)

def save_label(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(map(str, label)) + '\n')

def augment_and_save(image_path, label_path, save_image_path, save_label_path, transform, num_augmented):
    image = cv2.imread(image_path)
    labels = read_label(label_path)

    if labels.size == 0:  # Skip augmentation if no labels are found
        return

    class_labels = labels[:, 0]
    bboxes = labels[:, 1:]

    for i in range(num_augmented):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = transformed['image']
        augmented_bboxes = transformed['bboxes']

        if len(augmented_bboxes) == 0:  # Skip if no bounding boxes remain after augmentation
            continue

        augmented_labels = np.hstack((np.array(transformed['class_labels'])[:, np.newaxis], augmented_bboxes))

        augmented_image_path = os.path.join(save_image_path, f"aug_{i}_{os.path.basename(image_path)}")
        augmented_label_path = os.path.join(save_label_path, f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(augmented_image_path, augmented_image)
        save_label(augmented_label_path, augmented_labels)

# Paths to your dataset images and labels
dataset_image_path = f"{dataset.location}/train/images"
dataset_label_path = f"{dataset.location}/train/labels"
augmented_image_path = f"{dataset.location}/aug/images"
augmented_label_path = f"{dataset.location}/aug/labels"

os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Augment each image and its corresponding label in the dataset
for image_name in os.listdir(dataset_image_path):
    image_path = os.path.join(dataset_image_path, image_name)
    label_path = os.path.join(dataset_label_path, image_name.replace('.jpg', '.txt'))
    augment_and_save(image_path, label_path, augmented_image_path, augmented_label_path, transform, num_augmented=5)

In [ ]:
import shutil
import os

# Ensure the augmented directories exist
os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Copy original images to the augmented images directory
for image_name in os.listdir(dataset_image_path):
    src_image_path = os.path.join(dataset_image_path, image_name)
    dst_image_path = os.path.join(augmented_image_path, image_name)
    shutil.copy(src_image_path, dst_image_path)

# Copy original labels to the augmented labels directory
for label_name in os.listdir(dataset_label_path):
    src_label_path = os.path.join(dataset_label_path, label_name)
    dst_label_path = os.path.join(augmented_label_path, label_name)
    shutil.copy(src_label_path, dst_label_path)


import yaml
data = {
    'train': f"{dataset.location}/aug/images",
    'val': f"{dataset.location}/valid/images",
    'test': f"{dataset.location}/test/images",
    'nc': 2,
    'names': ['Fissure', 'Landslide']
}
with open(f"{dataset.location}/data.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

In [ ]:
from ultralytics import YOLO
from google.colab import files

# Load a pre-trained YOLOv8 model
model = YOLO("yolo11m.pt")

# Train the model on your Landslide dataset
model.train(
    data = f"{dataset.location}/data.yaml",  # Path to your data.yaml file
    epochs=50,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = f"{dataset.location}/runs/train",  # Directory to save training results
    name="landslide_detection",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)

# Download the saved model
files.download(f"{dataset.location}/runs/train/landslide_detection/weights/best.pt")

In [ ]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO(f"{dataset.location}/runs/train/landslide_detection/weights/best.pt")

# Evaluate the model using the val() method
metrics = model.val(data=f"{dataset.location}/data.yaml", split='test')

# Print evaluation metrics
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

In [ ]:
#from google.colab import files
#uploaded = files.upload()

#import os
#os.listdir('/content')

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')  # Set the locale to a UTF-8 compatible one

import subprocess
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install("onnx") # Installing onnx package

import torch
import onnx
import numpy as np
from google.colab import files

onnx_model_path = f"{dataset.location}/best.onnx"
torch.onnx.export(
    model,
    torch.randn(1, 3, 640, 640),  # Dummy input for the model
    onnx_model_path,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)

files.download(f"{dataset.location}/best.onnx")

In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics